In [1]:
import pandas as pd
import subprocess
from tqdm.auto import tqdm
import os
import shutil
from utils.functions import parse
from gensim.models.word2vec import Word2Vec
import numpy as np
import nltk
import torch
import json
from numpyencoder import NumpyEncoder

C:\Users\Grazia\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (5.1.0) doesn't match a supported version!
  RequestsDependencyWarning)
C:\Users\Grazia\anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
codes = pd.read_csv('../Datasets Code Snippets/post_with_only_code_snippets_final.csv')

In [75]:
codes

,id,code
0,37637877,void closeBuySell(int buyTicket){\n double b...
1,57596800,"#include <stdio.h>\n\nvoid sum(int *, int *, i..."
2,22667586,"#include ""buraco/game/board/model/KingChatFilt..."
3,22667586,"#include ""buraco\game/board/model\CardGroup.h""..."
4,55014685,#include <stdlib.h> /* EXIT malloc free */\n#i...
...,...,...
452666,64735134,"template <class T, class ...Ts>\nvoid printA(T..."
452667,65106538,#include <iostream>\nusing namespace std;\n\nv...
452668,25117355,#include <iostream>\nusing namespace std;\n\ni...
452669,55782153,#include<stdio.h>\n#include <limits.h>\n#inclu...


# Create CPG

In [ ]:
proc = subprocess.Popen(["echo", "%BASEDIR%"], stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
path_to_joern = out.decode('utf-8').split('\r')[0]

In [ ]:
def to_files(data_frame: pd.DataFrame, out_path):
    only_index = set([int(x.split('.')[0]) for x in os.listdir('data/code')])
    to_analyze = set(codes.index)-only_index

    for idx in tqdm(to_analyze):
        file_name = f"{idx}.cpp"
        if not os.path.exists(out_path + file_name):
            try:
                with open(out_path + file_name, 'w', encoding="utf-8") as f:
                    f.write(data_frame['code'][idx])
            except UnicodeEncodeError as e:
                print(e)
                print(idx)
                os.remove(out_path + file_name)
                break;

In [ ]:
to_files(codes, 'data/code/')

In [ ]:
subprocess.run([path_to_joern + "/joern-parse.bat", "data/code"],
                                      stdout=subprocess.PIPE, text=True, check=True)

# Embedding

In [5]:
codes = codes.rename(columns={'code':'tokens'})

codes.head()

,id,tokens
0,37637877,void closeBuySell(int buyTicket){\n double b...
1,57596800,"#include <stdio.h>\n\nvoid sum(int *, int *, i..."
2,22667586,"#include ""buraco/game/board/model/KingChatFilt..."
3,22667586,"#include ""buraco\game/board/model\CardGroup.h""..."
4,55014685,#include <stdlib.h> /* EXIT malloc free */\n#i...


In [9]:
for index in tqdm(range(codes.shape[0])):
    codes['tokens'][index] = parse.tokenizer(codes['tokens'][index])

C:\Users\Grazia\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
codes.to_csv("../Datasets Code Snippets/codes_snippets_tokenized.csv", index=False)

In [2]:
codes = pd.read_csv("../Datasets Code Snippets/codes_snippets_tokenized.csv")
print(codes.shape)
w2vmodel = Word2Vec(min_count=1)
w2vmodel.build_vocab(sentences=tqdm(codes.tokens))
w2vmodel.train(sentences=tqdm(codes.tokens), total_examples=w2vmodel.corpus_count, epochs=1)
print("Saving w2vmodel.")
w2vmodel.save(f"w2v_so/w2v_model")

(452671, 2)


C:\Users\Grazia\anaconda3\lib\site-packages\gensim\models\base_any2vec.py:618: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "



Saving w2vmodel.


# Creating input files

In [12]:
EDGE_TYPES = {
    "CONTROLS": 0,
    "DECLARES": 1,
    "DEF": 2,
    "DOM": 3,
    "FLOWS_TO": 4,
    "IS_AST_PARENT": 5,
    "IS_CLASS_OF": 6,
    "IS_FILE_OF": 7,
    "IS_FUNCTION_OF_AST": 8,
    "IS_FUNCTION_OF_CFG": 9,
    "POST_DOM": 10,
    "REACHES": 11,
    "USE": 12,
}

TYPE_MAP = {
    "AndExpression": 1,
    "Sizeof": 2,
    "Identifier": 3,
    "ForInit": 4,
    "ReturnStatement": 5,
    "SizeofOperand": 6,
    "InclusiveOrExpression": 7,
    "PtrMemberAccess": 8,
    "AssignmentExpression": 9,
    "ParameterList": 10,
    "IdentifierDeclType": 11,
    "SizeofExpression": 12,
    "SwitchStatement": 13,
    "IncDec": 14,
    "Function": 15,
    "BitAndExpression": 16,
    "UnaryExpression": 17,
    "DoStatement": 18,
    "GotoStatement": 19,
    "Callee": 20,
    "OrExpression": 21,
    "ShiftExpression": 22,
    "Decl": 23,
    "CFGErrorNode": 24,
    "WhileStatement": 25,
    "InfiniteForNode": 26,
    "RelationalExpression": 27,
    "CFGExitNode": 28,
    "Condition": 29,
    "BreakStatement": 30,
    "CompoundStatement": 31,
    "UnaryOperator": 32,
    "CallExpression": 33,
    "CastExpression": 34,
    "ConditionalExpression": 35,
    "ArrayIndexing": 36,
    "PostIncDecOperationExpression": 37,
    "Label": 38,
    "ArgumentList": 39,
    "EqualityExpression": 40,
    "ReturnType": 41,
    "Parameter": 42,
    "Argument": 43,
    "Symbol": 44,
    "ParameterType": 45,
    "Statement": 46,
    "AdditiveExpression": 47,
    "PrimaryExpression": 48,
    "DeclStmt": 49,
    "CastTarget": 50,
    "IdentifierDeclStatement": 51,
    "IdentifierDecl": 52,
    "CFGEntryNode": 53,
    "TryStatement": 54,
    "Expression": 55,
    "ExclusiveOrExpression": 56,
    "ClassDef": 57,
    "File": 58,
    "UnaryOperationExpression": 59,
    "ClassDefStatement": 60,
    "FunctionDef": 61,
    "IfStatement": 62,
    "MultiplicativeExpression": 63,
    "ContinueStatement": 64,
    "MemberAccess": 65,
    "ExpressionStatement": 66,
    "ForStatement": 67,
    "InitializerList": 68,
    "ElseStatement": 69,
    'ThrowStatement': 70,
    'CFGExceptionNode': 71,
    'CatchStatement': 72,
    'CatchList': 73
}

TYPE_MAP_OH = np.eye(len(TYPE_MAP))

In [13]:
def create_input_from_cpg(filepath, w2v, etypemap = EDGE_TYPES):
    try:
        nodes = pd.read_csv(filepath+"/nodes.csv", sep="\t", engine='python')
        edges = pd.read_csv(filepath+"/edges.csv", sep="\t", engine='python')
    except Exception as E:
        print(E)
        print(filepath)
        return [None, None]
    if len(nodes) == 0 or len(edges) == 0:
        print("Empty node / edge CSV")
        return [None, None]
    n, e = format_node_edges(nodes, edges)
    e = e[e.type != "IS_FILE_OF"]
    e = e[e.type.isin(etypemap)]
    
    node_id_dict = (
    n.reset_index(drop=True)
    .reset_index()[["key", "index"]]
    .set_index("key")
    .to_dict()["index"]
    )
    e = e[(e.start.isin(n.key)) & (e.end.isin(n.key))].copy()
    n.key = n.key.apply(lambda x: node_id_dict[x])
    e.start = e.start.apply(lambda x: node_id_dict[x])
    e.end = e.end.apply(lambda x: node_id_dict[x])
    
    n.code = n.code.fillna("")
    n.code = n.code.apply(embed_code, w2v=w2v).to_list()
    n.type = n.type.apply(one_hot_encode_type)
    n = n[n.type != -1]
    
    src = e["start"].to_numpy()
    dst = e["end"].to_numpy()
    nnodes = len(n)
    nfeat = torch.tensor([list(i.type) + list(i.code) for i in n.itertuples()]).float()
    etype = torch.tensor([etypemap[i] for i in e.type]).int()
    return n, e
    
    
    
def format_node_edges(n, e):
    """Format node and edges into appropriate input form."""
    nodes = n.copy()
    edges = e.copy()
    nodes.key -= 1
    edges.start -= 1
    edges.end -= 1
    node_key_type_map = nodes[["key", "type"]].set_index("key").to_dict()["type"]
    node_key_code_map = nodes[["key", "code"]].set_index("key").to_dict()["code"]
    edges["src"] = edges.start.apply(lambda x: node_key_type_map[x])
    edges["dest"] = edges.end.apply(lambda x: node_key_type_map[x])
    edges["src_feat"] = edges.start.apply(lambda x: node_key_code_map[x])
    edges["dest_feat"] = edges.end.apply(lambda x: node_key_code_map[x])
    return nodes, edges

def one_hot_encode_type(type_):
    try:
        return TYPE_MAP_OH[TYPE_MAP[type_] - 1].tolist()
    except:
        return -1
    
def embed_code(code: str, w2v):
    """Embed code using given word2vec model by averaging code embeddings."""
    code = nltk.word_tokenize(code.strip())
    if len(code) == 0:
        return np.zeros(100)
    wvecs = []
    for word in code:
        try:
            wvecs.append(w2v.wv[word])
        except:
            wvecs.append(np.zeros(100))
    return np.array(sum(np.array(wvecs)) / len(code), dtype="float32")

In [14]:
base_dir = "parsed/data/code/"
index_ben = 0
index_vul = 0
w2vmodel = Word2Vec.load("w2v_so/w2v_model")

all = [x.split('.')[0] for x in set(os.listdir('already_parsed'))]
already_created = [x.split('.')[0] for x in set(os.listdir('snippets_so_input'))]

to_create = set(all) - set(already_created)

for file in tqdm(to_create):
    #filename = file.split('.')[0]
    
    if not os.path.exists("snippets_so_input/eval/"+file+".txt"):
        flaw_loc = 0
        n, e = create_input_from_cpg(base_dir+file+".cpp", w2vmodel)

        if n is not None and e is not None:
            edges_list = list()
            targets = list()
            map_line = list()

            for index_edge in range(e.shape[0]):
                if index_edge in e.index:
                    type_edge = EDGE_TYPES[e['type'][index_edge]]

                    edges_list.append([e['start'][index_edge], type_edge, e['end'][index_edge]])


            targets.append([0])


            for index_node, row in n.iterrows():
                if pd.isnull(n['location'][index_node]):
                    map_line.append(0)
                else:
                    map_line.append(int(n['location'][index_node].split(':')[0]))


            dictionary = {"node_features": n.code.tolist(), "graph": edges_list, "targets": targets, 'map_line': map_line, 'index': [int(file), codes['id'][int(file)]]}

            with open("snippets_so_input/eval/"+file+".txt", 'w') as file:
                json.dump(dictionary, file, cls=NumpyEncoder)
                file.close()

Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node

Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node

Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node

Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node

Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node

...

Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node

Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node

Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node

Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node

Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV
Empty node

# Detecting vulnerabilities

In [2]:
%run VELVET\src\run_model.py snippets_so_input VELVET\src\config.yml -t True -x gnn -y transformer -p gnn/log.txt -q transformer/log.txt

Training with configuration: {'model': {'configuration': 'ggnn', 'base': {'hidden_dim': 256, 'dropout_rate': 0.1, 'num_edge_types': 24}, 'evaluation': {'top': 1, 'node_of_line': 7, 'window_size': 9}, 'ggnn': {'time_steps': [3, 1, 3, 1], 'residuals': {'1': [0], '3': [0, 1]}, 'add_type_bias': True}, 'transformer': {'ff_dim': 2048, 'num_layers': 6, 'attention_dim': 512, 'num_heads': 8}}, 'model_2': {'configuration': 'transformer', 'base': {'hidden_dim': 256, 'dropout_rate': 0.1, 'num_edge_types': 24}, 'evaluation': {'top': 1, 'node_of_line': 7, 'window_size': 9}, 'ggnn': {'time_steps': [3, 1, 3, 1], 'residuals': {'1': [0], '3': [0, 1]}, 'add_type_bias': True}, 'transformer': {'ff_dim': 2048, 'num_layers': 6, 'attention_dim': 512, 'num_heads': 8}}, 'data': {'max_batch_size': 1250, 'max_buffer_size': 100, 'max_node_size': 1024, 'valid_interval': 2500, 'max_valid_samples': 25000, 'max_token_length': 10, 'w2v_dimension': 256}, 'training': {'max_steps': 50, 'print_freq': 500, 'learning_rate': 

In [4]:
results_ml = pd.read_csv("Results/results_ml.csv")

In [5]:
results_ml

,index_code,id_post,is_vul,vul_line
0,100043,63708756,0,0
1,100072,58465449,0,0
2,100078,53143308,0,0
3,100083,51385050,0,0
4,100005,21285335,0,0
...,...,...,...,...
421019,97037,58740316,0,0
421020,98601,35509309,0,0
421021,94150,58920691,0,0
421022,98362,55820387,0,0


In [9]:
results_ml.loc[results_ml['is_vul']==1]

,index_code,id_post,is_vul,vul_line
17,100077,53143308,1,4
54,100087,63472782,1,15
64,100060,33241669,1,0
74,100123,49367727,1,38
140,10007,34442753,1,5
...,...,...,...,...
420973,99775,49182588,1,56
420980,9705,54427823,1,0
420982,98906,51591398,1,0
420994,95530,62056797,1,0
